In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
from torchvision.models.detection import SSD300_VGG16_Weights
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import os

In [9]:
train_labels=[]
test_labels=[]
train_imgs=[]
test_imgs=[]
classes=['Bear', 'Bull', 'Cat', 'Cattle',
           'Cheetah', 'Chicken', 'Crocodile',
           'Deer', 'Dog', 'Fox', 'Goat', 'Hippopotamus', 
           'Horse', 'Jaguar', 'Leopard', 'Lion', 'Lynx',
           'Monkey', 'Mule', 'Ostrich', 'Owl', 
           'Pig', 'Raccoon', 'Raven', 'Rhinoceros', 
           'Sheep', 'Snake', 'Tiger', 'Turkey', 'Turtle', 'Zebra']
"""classes=['Cat','Dog']"""
encode={}
for i in range(len(classes)):
    encode[classes[i]]=i+1

def checkclass(dirname):
    
    for animal in classes:
        if dirname.endswith(animal):
            return True
    return False

for dirname, dirs, filenames in os.walk('./data/'):
    
    if 'train' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        
        for label in sorted(os.listdir(labels)):
            
            
            if label.endswith('txt'):
                train_labels.append(labels+label)
        
        for image in sorted(filenames):
            
            
            if image.endswith('png') or image.endswith('jpg'):
                train_imgs.append(dirname+'/'+image)
    if 'test' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        for label in sorted(os.listdir(labels)):
            if label.endswith('.txt'):
                test_labels.append(labels+label)
        for image in sorted(filenames):
            if image.endswith('png'):
                test_imgs.append(dirname+'/'+image)


In [10]:
class AnimalDataset(object):
    def __init__(self, transforms,img_dir,label_dir):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = img_dir
        self.label=label_dir
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = self.imgs[idx]
        file_label = self.label[idx]
        img = Image.open(file_image).convert("RGB")
        target={}
        target['image_id']=torch.tensor([idx])
        labels=[]
        boxes=[]
        #Generate Label
        with open(file_label,'r')as f:
            lines=f.readlines()
            for line in lines:
                line_list=line.strip().split(' ')
                labels.append(encode[line_list[0]])
                boxes.append([float(i) for i in line_list[1:]])
                print(len(line_list))
            if self.transforms is not None:
                img=self.transforms(img)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)        
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        iscrowd = torch.zeros((len(classes),), dtype=torch.int64)

        target['boxes']=boxes
        target['labels']=labels
       
        target["area"] = area
        target["iscrowd"] = iscrowd
        return img, target

    def __len__(self):
        return len(self.imgs)